# Prototype Dataset
This file generates the path-to-path dataset to train a prototype transformer

The input output sequence of a logo should consist of 269 Parameters
- DeepSVG Embedding (256)
- Type "EOS" (1) (open for discussion, but it makes the most sense in the predicting process ~Corni)
- Type (6)
- Param (6)

Input Sequence
- In the input sequence, the last 13 Parameters are set to zero
- In a final model, these parameters are aimed to be left out (as context vector isn't based on that)

Output Sequence
- The Output sequence consists of the selected paths represented by their DeepSVG Embedding
- The Animation is represented through the type and the parameters.
- Each output sequence ends with an EOS Token, where the EOS-type in the one-hot-encoded part is set to 1.

# Import DeepSVG Embedding

# Build Input Sequence (Embedded Logo Paths)

# Import Old Dataset

In [ ]:
import pickle
import pandas as pd

# Load the pickle file
with open("data/surrogate_model/animation_label.pkl", "rb") as f:
    imported_pickle = pickle.load(f)

# Create a DataFrame from the imported data
filtered_data = pd.DataFrame(imported_pickle, columns=["file", "animation_id", "model_output", "label"])

# Define the mapping for ratings
mapping_dict = {"Very Good": 6, "Good": 5, "Bad": 4, "Okay": 3, "Very Bad": 2, "no_rating": 1}

# Replace the 'label' column with the corresponding ratings using .map
filtered_data['rating'] = filtered_data['label'].map(mapping_dict)

# Extract unique logos by splitting 'file' with "_animation" and using .str.get(0)
logos = filtered_data['file'].str.split('_animation').str.get(0).unique()

In [ ]:
#example
filtered_data[filtered_data["file"].str.contains("logo_1_")]

In [ ]:
logos

In [ ]:
# create a data frame for the collected best animations
best_output = pd.DataFrame()

# go through each logo to find the best animation
for logo in logos:

    # make a data frame that contains all the animations of one logo
    temp = filtered_data[filtered_data["file"].str.contains(logo)]

    # create a sum 
    mean_by_label = temp.groupby('file')['rating'].mean().reset_index()

    best_logo = mean_by_label.loc[mean_by_label['rating'].idxmax()]

    # get all the animated paths with the best animation of the logo
    best_animations2 = temp[temp["file"]==best_logo["file"]]

    # add to the file
    best_output = pd.concat([best_output,best_animations2],axis=0, ignore_index=True)
best_output = best_output.sort_values(by=['file','animation_id'])

In [ ]:
best_output

# Build Output Sequence (Embedded Paths with Animation Vector)